# Algorithms

## General code

In [1]:
from threading import Thread, Lock, Barrier
from typing import Any
from copy import copy
from random import randint
from time import sleep

These are just the libraries in use

In [2]:
class Message:
    def __init__(self, content:Any, nodeId:int):
        self.content = content
        self.nodeId = nodeId

message has a content field, it can be anything

and a nodeId field that signifies the sender node

In [3]:
class Network:
    lock:Lock = Lock()

    def create(self, n:int, f:int, nodeClass) -> None:
        self.n:int = n
        self.f:int = f
        self.nodes:dict[list[Message] | Node] = {i:{
            "object":nodeClass(self, i),
            "received":[]
        } for i in range(n)}
        for node in [data["object"] for data in self.nodes.values()]:
            node.start()
    
    def send(self, receiverId:int, message:Message) -> None:
        self.lock.acquire()
        if not receiverId in self.nodes.keys():
            self.nodes[receiverId] = {"received":[]}
        self.nodes[receiverId]["received"].append(copy(message))
        self.lock.release()

    def receive(self, nodeId:int) -> Message | None:
        if len(self.nodes[nodeId]["received"]) == 0:
            return None
        else:
            self.lock.acquire()
            message = self.nodes[nodeId]["received"].pop(0)
            self.lock.release()
            return message
        
    def receiveFrom(self, receiverId:int, senderId:int) -> Message | None:
        self.lock.acquire()
        if not receiverId in self.nodes.keys():
            self.nodes[receiverId] = {"received":[]}
        for message in self.nodes[receiverId]["received"]:
            if message.nodeId == senderId:
                self.nodes[receiverId]["received"].remove(message)
                self.lock.release()
                return message
        self.lock.release()
        return None
    
    def multicast(self, group:list[int], message:Message) -> None:
        for receiverId in group:
            self.send(receiverId, message)

    def wait(self):
        while True:
            if all([not data["object"].running for data in self.nodes.values()]):
                return

class Node(Thread):
    def __init__(self, network:Network, nodeId:int):
        self.network:Network = network
        self.nodeId:int = nodeId
        self.running = True
        super().__init__()


The purpose of the classes above is to have a unified network and node class between algorithms

## Phase-King Algorithm

In [4]:
class PhaseKing(Node):
    def run(self):
        v = Message(randint(0, 10), self.nodeId)
        for phase in range(0, self.network.n):
            # Round 1
            self.network.multicast(range(self.network.n), v)
            vs = []
            for _ in range(self.network.n):
                while True:
                    message = self.network.receive(self.nodeId)
                    if message is not None:
                        vs.append(message)
                        break
            mult = 0
            for value in vs:
                count = vs.count(value)
                if count > mult:
                    mult = count
                    v = value
            if mult < self.network.n/2:
                v = v
            
            # Round 2
            if self.nodeId == phase:
                v.nodeId = self.nodeId
                print(f'{self.nodeId} is king')
                self.network.multicast(range(self.network.n), v)
            while True:
                message = self.network.receiveFrom(self.nodeId, phase)
                if message is not None:
                    v_k = message
                    break
            if mult <= (self.network.n)/2 + self.network.f:
                v = v_k
            
        print(f'value: {v.content}')
        self.v = v # save the value
        self.running = False

The above code is for the Phase-King algorithm

In [5]:
network = Network()
network.create(4, f=1, nodeClass=PhaseKing)

network.wait()

print(f"is there consensus? {all([data['object'].v.content == network.nodes[0]['object'].v.content for data in network.nodes.values()])}")

0 is king
1 is king
2 is king
3 is king
value: 7
value: 7
value: 7
value: 7
is there consensus? True


Above we're testing the phase-king algorithm, and as shown it gets to a consensus

## Replication

In [6]:
import json

class RobustDatabase(Node):

    def __init__(self, network, id):
        self.running = True
        self.database = {}
        super().__init__(network, id)

    def run(self):
        while self.running:
            message = self.network.receive(self.nodeId)
            if message is not None:
                id = message.nodeId
                data = message.content
                match data["mode"]:
                    case "sync":
                        self.database[data["key"]] = data["value"]
                    case "addData":
                        self.database[data["key"]] = data["value"]
                        message.content["mode"] = "sync"
                        self.network.multicast(range(self.network.n), message)
                        self.network.send(id, Message(None, self.nodeId))
                    case _:
                        self.network.send(id, Message(self.database[data["key"]], self.nodeId))
        print(f"Node {self.nodeId} finished execution")

    def crash(self):
        self.running = False

In [7]:
class App:
    def __init__(self):
        self.network = Network()
        self.network.create(10, None, RobustDatabase)

    def getResource(self, key, nodeId) -> Any:
        counter = 0
        finalCounter = 0
        myMessage = Message({"mode":"", "key":key}, -1)
        self.network.send(nodeId, myMessage)
        while True:
            message = self.network.receiveFrom(-1, nodeId)
            if message is not None:
                return message.content
            sleep(1)
            if counter == 10:
                nodeId += 1 if nodeId < self.network.n else 0
                self.network.send(nodeId, myMessage)
                counter = 0
                finalCounter += 1
            if finalCounter == 10:
                return None

    def putResource(self, key, value, nodeId) -> None:
        myMessage = Message({"mode":"addData", "key":key, "value":value}, -1)
        self.network.send(nodeId, myMessage)
        while self.network.receiveFrom(-1, nodeId) is None: pass

app = App()
print("initialized app")
app.putResource("test", "test", 0)
print("put resource test:test at node 0")
print(f'Got resource at test at node {1}: {app.getResource("test", 1)}')
n = randint(0, app.network.n)
print(f"Crashing node {n}")
app.network.nodes[n]["object"].crash()
n = randint(0, app.network.n)
print(f"Crashing node {n}")
app.network.nodes[n]["object"].crash()
print(f'Got resource at test at node {n}: {app.getResource("test", n)}')

initialized app
put resource
Got resource at test at node 1: test
Crashing node 0
Crashing node 4
Node 4 finished execution
Node 0 finished execution
Got resource at test at node 4: test


## Blockchains

In [8]:
from time import time
from hashlib import sha256

class HashPointer:
    def __init__(self, address, data):
        self.address = address
        self.hash = sha256(data.encode()).hexdigest()


class Block(Node):
    def __init__(self, network, nodeId, data):
        super().__init__(network, nodeId)
        addr = self.nodeId+1 if self.nodeId < self.network.n else 0
        self.data = data
        self.HP = HashPointer(addr, data)


